<h1 id="header">BookMe Company</h1>

### Supervised Project

In [2]:
# Relevant Package Importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
import scipy.stats as stats
from scipy.stats import chi2_contingency
# Scaling
from sklearn.preprocessing import MinMaxScaler
# Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# Model Assessment
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

### Pre-Processing


In [22]:
#Import Data from cluster.csv
db = pd.read_csv('train.csv')
db[db.duplicated(keep = False)] # We don't have duplicates

Empty DataFrame
Columns: [Cust_ID, Churn, Name, Longevity, Year_Birth, TypeTravel, RoomType, RewardPoints, Comfort, ReceptionSchedule, FoodDrink, Location, Wifi, Amenities, Staff, OnlineBooking, PriceQuality, RoomSpace, CheckOut, Checkin, Cleanliness, BarService]
Index: []

[0 rows x 22 columns]

##### There aren't any duplicates in our data

##### Set Customer ID as Index

In [23]:
db.set_index('Cust_ID', inplace = True)

##### Devide Target from data

In [24]:
data = db.drop(['Churn'], axis=1)
target = db['Churn']

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15589 entries, 1 to 15589
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               15589 non-null  object 
 1   Longevity          15589 non-null  object 
 2   Year_Birth         15394 non-null  float64
 3   TypeTravel         15589 non-null  object 
 4   RoomType           15589 non-null  object 
 5   RewardPoints       15589 non-null  int64  
 6   Comfort            15589 non-null  int64  
 7   ReceptionSchedule  15589 non-null  int64  
 8   FoodDrink          15589 non-null  int64  
 9   Location           15589 non-null  int64  
 10  Wifi               15589 non-null  int64  
 11  Amenities          15589 non-null  int64  
 12  Staff              15589 non-null  int64  
 13  OnlineBooking      15589 non-null  int64  
 14  PriceQuality       15589 non-null  int64  
 15  RoomSpace          15589 non-null  int64  
 16  CheckOut           155

Data types:
- We'll need to change the data types of all the int64 to int4 or int8 after normalization
- We have nulls in the year of birth variable

##### Separate Numerical from Categorical Data

In [55]:
num_var = data.select_dtypes(include=np.number).set_index(data.index).columns
car_var = data.select_dtypes(exclude=np.number).set_index(data.index).columns
print(num_var)
print(cat_var)

Index(['Year_Birth', 'RewardPoints', 'Comfort', 'ReceptionSchedule',
       'FoodDrink', 'Location', 'Wifi', 'Amenities', 'Staff', 'OnlineBooking',
       'PriceQuality', 'RoomSpace', 'CheckOut', 'Checkin', 'Cleanliness',
       'BarService'],
      dtype='object')
Index(['Name', 'Longevity', 'TypeTravel', 'RoomType'], dtype='object')


##### Replace incorrect performance values

In [27]:
data[num_var].describe()

Year_Birth  RewardPoints       Comfort  ReceptionSchedule  \
count  15394.000000  15589.000000  15589.000000       15589.000000   
mean    1981.706444   5022.593816      2.841619           2.997242   
std       15.179042   1027.962379      1.388624           1.518994   
min     1936.000000    409.000000      0.000000           0.000000   
25%     1970.000000   4445.000000      2.000000           2.000000   
50%     1981.000000   5088.000000      3.000000           3.000000   
75%     1994.000000   5649.000000      4.000000           4.000000   
max     2014.000000   6950.000000      5.000000           5.000000   

          FoodDrink      Location          Wifi     Amenities         Staff  \
count  15589.000000  15589.000000  15589.000000  15589.000000  15589.000000   
mean       2.844570      2.986016      3.245109      3.374816      3.506383   
std        1.436948      1.299438      1.327026      1.352417      1.319565   
min        0.000000      1.000000      0.000000      0.000000      1.000000   
25%        2.000000      2.000000      2.000000      2.000000      3.000000   
50%        3.000000      3.000000      3.000000      4.000000      4.000000   
75%        4.000000      4.000000      4.000000      4.000000      5.000000   
max        5.000000      5.000000      6.000000      5.000000      5.000000   

       OnlineBooking  PriceQuality     RoomSpace      CheckOut       Checkin  \
count   15589.000000  15589.000000  15589.000000  15589.000000  15589.000000   
mean        3.454231      3.459683      3.470845      3.700558      3.327282   
std         1.310343      1.268130      1.293873      1.158644      1.266872   
min         0.000000      1.000000      0.000000      1.000000      1.000000   
25%         2.000000      3.000000      2.000000      3.000000      3.000000   
50%         4.000000      4.000000      4.000000      4.000000      3.000000   
75%         5.000000      4.000000      5.000000      5.000000      4.000000   
max         5.000000      5.000000      5.000000      5.000000      5.000000   

        Cleanliness    BarService  
count  15589.000000  15589.000000  
mean       3.692347      3.347360  
std        1.154437      1.300452  
min        1.000000      0.000000  
25%        3.000000      2.000000  
50%        4.000000      3.000000  
75%        5.000000      4.000000  
max        5.000000      5.000000

In [53]:
print((data.loc[:, 'Comfort':] == 0).sum().sum())
print((data.loc[:, 'Comfort':] == 1).sum().sum())
print((data.loc[:, 'Comfort':] == 2).sum().sum())
print((data.loc[:, 'Comfort':] == 3).sum().sum())
print((data.loc[:, 'Comfort':] == 4).sum().sum())
print((data.loc[:, 'Comfort':] == 5).sum().sum())
print((data.loc[:, 'Comfort':] == 6).sum().sum())

2525
25447
34395
45349
62009
48485
36


About valuation of Customer Satisfaction
- 0.01% of valuation 6 --> To remove the 6
- 1.2% of valuation 0 --> seams legit. Keep the 0

##### Transformations to be made
- Remove 6 from Wifi Valuations
- Transform all 'y' to 'yes' in Longevity
- Create bins from Reward Points
- Create variable Gender from the name variable & Drop name variable

##### Make transformations onle after separating training from validation

In [11]:
data[num_var]['Wifi'] = data[num_var]['Wifi'].replace(6,5)
data[car_var]['Longevity'] = data[car_var]['Longevity'].replace('y','yes')

In [ ]:
data[num_var]['RewardPoints_Bins'] = pd.cut(data[num_var]['RewardPoints'], 4, labels=['Bronze', 'Silver', 'Gold', 'Platinum'])
data[num_var]['RewardPoints_Bins'] = data[num_var]['RewardPoints_Bins'].replace('Bronze',0).replace('Silver',1).replace('Gold',2).replace('Platinum',3)
data[num_var].drop(columns = ['RewardPoints'], axis=1, inplace=True)

In [59]:
gender = data[car_var]['Name'].str.split(' ',expand = True,n=1)[0]
gender

Cust_ID
1        Ms.
2        Mr.
3        Mr.
4        Ms.
5        Mr.
        ... 
15585    Ms.
15586    Ms.
15587    Mr.
15588    Mr.
15589    Mr.
Name: 0, Length: 15589, dtype: object

In [56]:
gender = data[car_var]['Name'].str.split(' ',expand = True,n=1)[0]
data[car_var]['Gender'] = gender
data[car_var]['Gender'] = data[car_var]['Gender'].replace('Ms.',0).replace('Mr.',1)
data[car_var].drop(columns = ['Name'], axis = 1, inplace = True)

# gender = test['Name'].str.split(' ',expand = True,n=1)[0]
# test['Gender'] = gender
# test['Gender'] = test['Gender'].replace('Ms.',0).replace('Mr.',1)
# test.drop(columns = ['Name'], axis = 1, inplace = True)

KeyError: 'Gender'

In [ ]:
# from datetime import date
# db['Age'] = date.today().year - db['Year_Birth']
# db.drop(columns = ['Year_Birth'], axis=1, inplace=True)
# test['Age'] = date.today().year - test['Year_Birth']
# test.drop(columns = ['Year_Birth'], axis=1, inplace=True)

In [ ]:
# Fill year of brith NAs with median
db['Year_Birth'] = db['Year_Birth'].fillna(db['Year_Birth'].median()).astype(int).astype(np.int64)

In [13]:
# Converting every column that is int64 to int8 for performance issues
db.iloc[:,(db.dtypes=='int64').values] = db.iloc[:,(db.dtypes=='int64').values].astype(np.int16)
test.iloc[:,(test.dtypes=='int64').values] = test.iloc[:,(test.dtypes=='int64').values].astype(np.int16)

##### Change Year of Birth with Date

In [14]:
# from datetime import date
# db['Age'] = date.today().year - db['Year_Birth']
# db.drop(columns = ['Year_Birth'], axis=1, inplace=True)
# test['Age'] = date.today().year - test['Year_Birth']
# test.drop(columns = ['Year_Birth'], axis=1, inplace=True)

In [16]:
#db.describe().T

##### Notes from initial pre-processing
- We have a minimum age of 8 years and max of 81 in the test file
- Min 8 and max 86 in the train file

##### We should't make new variables, dummy variables, nor replace categorical values for numerical, since we need the end variables to be the same as the test variables

----
##### Start Division between Data and Target

In [21]:
#print(target.info())
#print(data.info())

##### If we want to use Hold-out method

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_val, y_train, y_val = train_test_split(data, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=15, 
                                                    shuffle=True, 
                                                    stratify=target
                                                   )

In [24]:
print('train:{}% | validation:{}%'.format(round(len(y_train)/len(target),2),
                                                     round(len(y_val)/len(target),2)
                                                    ))

train:0.8% | validation:0.2%


##### Dividing DataSet between Numerical and Categorical data

In [25]:
# Numerical data
X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)
X_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)

# Categorical data
X_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)
X_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)

##### Scaling numerical data between 0 and 1

In [26]:
scaler = MinMaxScaler()
scaler.fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num) # this will return an array
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns, index = X_train.index) # Convert the array to a pandas dataframe
X_train_num_scaled.head(5)

Year_Birth  RewardPoints  Comfort  ReceptionSchedule  FoodDrink  \
Cust_ID                                                                    
9587       0.846154      0.508026      0.6                0.4        0.6   
4281       0.551282      0.695459      0.2                0.2        0.2   
6256       0.576923      0.153340      0.6                0.6        0.6   
10584      0.358974      0.941752      0.6                1.0        1.0   
9339       0.384615      0.912399      1.0                1.0        1.0   

         Location  Wifi  Amenities  Staff  OnlineBooking  PriceQuality  \
Cust_ID                                                                  
9587         0.50   0.8        0.8   0.75            0.8          0.75   
4281         0.75   0.2        0.2   0.00            0.6          1.00   
6256         0.50   0.6        0.2   0.00            0.6          1.00   
10584        1.00   0.2        0.6   0.75            0.6          0.50   
9339         1.00   1.0        1.0   0.75            1.0          1.00   

         RoomSpace  CheckOut  Checkin  Cleanliness  BarService  
Cust_ID                                                         
9587           0.8      1.00     0.50         0.75         0.8  
4281           0.4      0.75     0.00         0.75         0.2  
6256           0.8      0.75     0.00         1.00         0.2  
10584          0.6      0.50     0.50         0.50         0.6  
9339           1.0      1.00     0.75         1.00         0.6

In [27]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns, index = X_val.index)
X_val_num_scaled.head(5)

Year_Birth  RewardPoints  Comfort  ReceptionSchedule  FoodDrink  \
Cust_ID                                                                    
6798       0.346154      0.423024      0.6                0.8        0.8   
7661       0.564103      0.753096      0.4                0.4        0.4   
5965       0.602564      0.735514      0.6                0.6        0.4   
1171       0.974359      0.689344      0.2                0.8        0.2   
11840      0.756410      0.604953      0.4                0.4        0.4   

         Location  Wifi  Amenities  Staff  OnlineBooking  PriceQuality  \
Cust_ID                                                                  
6798         0.75   0.6        0.6   0.50            0.6          0.50   
7661         0.25   0.8        1.0   1.00            0.8          0.75   
5965         0.50   1.0        0.8   1.00            0.6          0.50   
1171         0.50   0.4        0.2   0.25            0.4          1.00   
11840        0.75   0.8        0.4   0.75            0.8          0.50   

         RoomSpace  CheckOut  Checkin  Cleanliness  BarService  
Cust_ID                                                         
6798           0.6      0.50     0.25         0.50         0.6  
7661           0.8      0.75     0.75         0.75         0.6  
5965           0.6      0.50     0.50         0.50         1.0  
1171           0.8      0.75     0.75         0.75         0.4  
11840          1.0      0.75     0.75         1.00         0.8

### Feature Selection

----

#### Apply Logistic Regression

In [77]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

In [ ]:
print('Train:', log_model.score(X_train, y_train))
print('Validation:', log_model.score(X_val, y_val))

----

##### K-Fold Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression